In [230]:
import time
import pickle
import random
from gensim.models import doc2vec
import numpy as np
import time 

import matplotlib.pyplot as plt
from collections import defaultdict

from sklearn.cluster import DBSCAN

from crashsimilarity import other_distances as od
from crashsimilarity.downloader import SocorroDownloader
from crashsimilarity import utils
from crashsimilarity import tmp
from crashsimilarity.distance import DistanceCalculator

In [231]:
vocab = pickle.load(open('data/compressed_vocab.pickle', 'rb'))
pos2vocab = dict([(i[1], i[0]) for i in vocab.items()])
len(vocab)

222895

In [52]:
model = doc2vec.Doc2Vec.load('data/new-model/model_dm_200_30_days.model')
model

INFO:gensim.utils:loading Doc2Vec object from data/new-model/model_dm_200_30_days.model
INFO:gensim.utils:loading wv recursively from data/new-model/model_dm_200_30_days.model.wv.* with mmap=None
INFO:gensim.utils:loading syn0 from data/new-model/model_dm_200_30_days.model.wv.syn0.npy with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:loading docvecs recursively from data/new-model/model_dm_200_30_days.model.docvecs.* with mmap=None
INFO:gensim.utils:loading doctag_syn0 from data/new-model/model_dm_200_30_days.model.docvecs.doctag_syn0.npy with mmap=None
INFO:gensim.utils:loading syn1neg from data/new-model/model_dm_200_30_days.model.syn1neg.npy with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:loaded data/new-model/model_dm_200_30_days.model


In [232]:
multiple_sig = pickle.load(open('data/multiple_sig_last_year.pickle', 'rb'))
prepared = [sig['prepared'] for sig in multiple_sig]
len(multiple_sig)

261

In [233]:
groups = []
corpus = []
for p in prepared:
    group = []
    for t in p:
        corpus.append(t)
        group.append(len(corpus)-1)
    groups.append(group)
compressed_corpus = [[str(vocab.get(i, i)) for i in c] for c in corpus]
compressed_corpus = [doc2vec.TaggedDocument(trace, [i]) for i, trace in enumerate(compressed_corpus)]
groups = [g for g in groups if len(g) > 1]

In [234]:
candidates = []
for c in corpus:
    ok = True
    qq = set()
    for t in c:
        if '@0x' in t:
            ok = False
        qq.add(t)
    if ok and len(qq) > len(c) / 2.0 and len(c) > 10 and c not in candidates:
        candidates.append(c)
len(candidates)

221

In [44]:
class Tmp(object):
    def __init__(self, words):
        self.words = words
def struct_dist_calc(s1, s2):
    return od.edit_distance2(s1, s2, subst_cost=od.struct_word_dist)

In [9]:
# candidates_struct_dist = od.distance_matrix([Tmp(i) for i in candidates], struct_dist_calc)
candidates_struct_dist = pickle.load(open('candidates_struct_dist', 'rb'))

In [10]:
candidates_struct_dist.shape

(298, 298)

In [45]:
def lcs_f(s1, s2):
    return len(tmp.lcs(s1, s2))

In [46]:
candidates_lcs = od.distance_matrix([Tmp(i) for i in candidates], lcs_f)

10.0%, 1.1770076751708984 s.
20.0%, 2.679997682571411 s.
30.0%, 4.053704500198364 s.
40.0%, 5.143387317657471 s.
50.0%, 6.347704172134399 s.
60.0%, 7.663520812988281 s.
70.0%, 8.677368402481079 s.
80.0%, 10.07191276550293 s.
90.0%, 11.303510665893555 s.


In [145]:
from pprint import pprint

In [200]:
def generate(trace, count=10, percent=0.33):
    rv = []
    idx = []
    new = int(len(trace) * percent)
    for i in range(count):
        cur = list(set([random.randint(0, len(trace)-1) for _ in range(new)]))
        idx.append(cur)
    # pprint(idx)
    rv = [change(trace, i) for i in idx]
    return rv

In [199]:
def change(trace, idx):
    other = trace[:]
    for i in idx:
        t = trace[i]
        if t in vocab:
            similar = model.most_similar(str(vocab[t]))
            for w, _ in similar:
                x = pos2vocab[int(w)]
                if not x.endswith('________'):
                    other[i] = x
                    break
        else:
            other[i] = x
    return other

In [104]:
len(candidate)

25

In [115]:
idx = generate(candidate, count=5)

In [47]:
__tmp = []
for i in range(candidates_lcs.shape[0]):
    for j in range(candidates_lcs.shape[1]):
        if j <= i:
            continue
        if candidates_lcs[i, j] > 0 and candidates_lcs[i, j] > max(len(corpus[i]), len(corpus[j])) / 2.0:
            __tmp.append((i,j, candidates_lcs[i, j]))
len(__tmp)

812

In [191]:
candidates[24]

['ns_abort_oom',
 'nsacstring_internal::assign',
 "mozilla::dom::workers::`anonymous namespace'::fetcheventrunnable::handlebodywithheaders",
 "mozilla::dom::workers::`anonymous namespace'::fetcheventrunnable::init",
 'mozilla::dom::workers::serviceworkerprivate::sendfetchevent',
 "mozilla::dom::workers::`anonymous namespace'::continuedispatchfetcheventrunnable::run",
 'mozilla::net::httpbasechannel::ensureuploadstreamiscloneablecomplete',
 'mozilla::detail::runnablemethodimpl<t>::run',
 'nsthread::processnextevent',
 'mozilla::ipc::messagepump::run',
 'messageloop::runhandler',
 'nsbaseappshell::run',
 'nsappshell::run',
 'nsappstartup::run',
 'xremain::xre_mainrun',
 'xremain::xre_main',
 'xre_main']

In [192]:
xxx  = generate(candidates[24], count=5)

[[0, 9, 11, 5], [16, 2, 13], [16, 2, 5, 7], [4, 5, 6, 9, 15], [8, 12, 13, 7]]


In [195]:
centers_count = 10
centers = list(set([random.randint(0, len(candidates)) for _ in range(centers_count)]))
centers

[98, 68, 74, 43, 138, 206, 111, 178, 85, 216]

In [203]:
clusters = [generate(candidates[i], count=5) for i in centers]

In [202]:
clusters[0][0] == clusters[0][1]

False

In [204]:
corpus = []
groups = []
i = 0
for c in clusters:
    g = []
    for x in c:
        corpus.append(x)
        g.append(i)
        i += 1
    groups.append(g)
len(corpus)

50

In [249]:
len(corpus)

755

In [247]:
ced = od.distance_matrix([Tmp(i) for i in corpus], od.edit_distance2)

9.999824336430867%, 22.328802585601807 s.
19.999648672861735%, 40.243231773376465 s.
29.999473009292604%, 63.4761221408844 s.
39.99929734572347%, 81.24119281768799 s.
49.99912168215434%, 100.24409294128418 s.
59.99894601858521%, 116.15589809417725 s.
69.99877035501608%, 132.38292479515076 s.
79.99859469144694%, 150.78666996955872 s.
89.99841902787782%, 167.29236268997192 s.
99.99824336430868%, 183.36088466644287 s.


In [206]:
csd = od.distance_matrix([Tmp(i) for i in corpus], struct_dist_calc)

9.959183673469388%, 0.2979896068572998 s.
19.918367346938776%, 0.502507209777832 s.
29.877551020408163%, 0.6331648826599121 s.
39.83673469387755%, 0.8284823894500732 s.
49.795918367346935%, 1.1091279983520508 s.
59.755102040816325%, 1.2701685428619385 s.
69.71428571428571%, 1.4290871620178223 s.
79.6734693877551%, 1.5999040603637695 s.
89.63265306122449%, 1.749948263168335 s.
99.59183673469387%, 1.8994369506835938 s.


In [211]:
def dbscan(dist, eps=0.5, min_samples=5):
    db = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(dist)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    return labels, n_clusters, core_samples_mask

In [248]:
l,n, c = dbscan(ced, eps = 9, min_samples=2)
n

80

In [227]:
groups

[[0, 1, 2, 3, 4],
 [5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24],
 [25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34],
 [35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44],
 [45, 46, 47, 48, 49]]

In [244]:
l

array([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1, -1, -1, -1,  2,  2,  3,  3,
        3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8, -1,  8,  8,  8])

In [246]:
labels_to_clusters(l)

defaultdict(list,
            {0: [0, 1, 2, 3, 4],
             1: [5, 6, 7, 8, 9],
             2: [13, 14],
             3: [15, 16, 17, 18, 19],
             4: [20, 21, 22, 23, 24],
             5: [25, 26, 27, 28, 29],
             6: [30, 31, 32, 33, 34],
             7: [35, 36, 37, 38, 39, 40, 41, 42, 43, 44],
             8: [45, 47, 48, 49]})

In [245]:
def labels_to_clusters(labels):
    clusters = defaultdict(list)
    for i, c in enumerate(labels):
        if c != -1:
            clusters[c].append(i)
    return clusters

In [76]:
other = []
for t in candidate:
    if t in vocab:
        similar = model.most_similar(str(vocab[t]))
        for w, _ in similar:
            x = pos2vocab[int(w)]
            if not x.endswith('________'):
                other.append(x)
                break
    else:
        other.append(t)